In [1]:
pip install kubernetes

Note: you may need to restart the kernel to use updated packages.


In [161]:
class K8SClient(object):
    def __init__(self):
        self.client = dynamic.DynamicClient(
            api_client.ApiClient(configuration=config.load_incluster_config())
        )

    @staticmethod
    def manifest_base(configmap_name):
        configmap_manifest = {
            "kind": "ConfigMap",
            "apiVersion": "v1",
            "metadata": {"name": configmap_name},
            "data": {}
        }
        return configmap_manifest

    @staticmethod
    def set_location(
        path,
        svc,
        port
    ):
        this_location = nginx.Location('/'+ f'{path}' + '/',
                                       nginx.Key('rewrite' ,'/'+ f'{path}' + '/(.*)  /$1 break'),
                                       nginx.Key('proxy_pass', f'http://{svc}:{port}')
                                      )
        return this_location
        
        
    def get_configmap(
        self,
        config_map_name,
        namespace="default"
    ):
        result = self.api.get(
            name=config_map_name,
            namespace=namespace
        )
        return result

    def patch_configmap(
        self,
        config_map_name,
        configmap_manifest,
        namespace="default"
    ):
        update_configmap = api.patch(
            name=config_map_name,
            namespace=namespace,
            body=configmap_manifest
        )

    def create_nginx_conf(
        self,
        configmap_name,
        list_of_svcs
    ):
        configmap_manifest = self.manifest_base(configmap_name)
        nginx_conf = nginx.Conf()
        nginx_server = nginx.Server()
        nginx_server.add(nginx.Key('listen', '80'))
        for svc_details in list_of_svcs:
            nginx_server.add(self.set_location(
                svc_details['path'],
                svc_details['svc'],
                svc_details['port']
            ))
        nginx_conf.add(nginx_server)
        configmap_manifest["data"]["nginx.conf"] = " ".join(nginx_conf.as_strings)
        self.patch_configmap(
            config_map_name=configmap_name,
            configmap_manifest=configmap_manifest
        )
        return nginx_conf

In [162]:
configmap_name = "nginx-conf"

In [163]:
k = K8SClient()

In [164]:
res = k.create_nginx_conf(
    configmap_name="nginx-conf",
    list_of_svcs=[
    {"svc": "httpbin.default.svc.cluster.local", "port": 5000, "path": "httpbin"},
    {"svc": "nodeserver.default.svc.cluster.local", "port": 5000, "path": "nodeserver"}
]
)